In [43]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


import numpy as np

In [45]:
ALL_GAMES = pd.read_csv('../data/raw/all_games_0626.csv')
ALL_GAMES['total_points'] = ALL_GAMES['home_team_score'] + ALL_GAMES['away_team_score']
ALL_GAMES = ALL_GAMES[ALL_GAMES.game_quarter < 5] ##TODO include overtimes
ALL_GAMES['point_start_time'] = 720 - ALL_GAMES['point_start_time']

/var/folders/r4/cqt7s71n4k18qp94swd6_njr0000gn/T/ipykernel_86231/1311389087.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  ALL_GAMES = pd.read_csv('../data/raw/all_games_0626.csv')


In [46]:
features = ['thrower_x', 'thrower_y', 'start_on_offense', 'point_start_time', 'possession_num', 'possession_throw', 'game_quarter', 'quarter_point', 'is_home_team', 'home_team_score', 'away_team_score', 'gameID', 'total_points']
ALL_GAMES = ALL_GAMES[features]

In [ ]:
for gameID in ALL_GAMES.gameID.unique():
    GAME = ALL_GAMES[ALL_GAMES.gameID == gameID]
    

In [72]:
GAME = ALL_GAMES[ALL_GAMES.gameID == ALL_GAMES.gameID[0]]

In [232]:
games = []
for gameID in ALL_GAMES.gameID.unique():
    GAME = ALL_GAMES[ALL_GAMES.gameID == gameID]
    points = []
    prev_point_time = 720
    current_quarter = 1
    prev_df = None
    error_points = 0
    for group_keys, group_df in GAME.groupby(['total_points', 'game_quarter']):
        if prev_df is None:
            prev_df = group_df
            continue

        prev_point_time = prev_df.point_start_time.max()
        if (current_quarter != group_df.game_quarter.max()):
            current_quarter = group_df.game_quarter.max()
            current_point_time = 0
        else:
            current_point_time = group_df.point_start_time.max()
        if current_point_time >= prev_point_time:
            error_points = error_points + 1
            continue
        times = np.linspace(prev_point_time, current_point_time + 1, len(prev_df))
        prev_df['times'] = times
        points.append(prev_df)
        prev_df = group_df
        if group_keys == (GAME.home_team_score.max()+GAME.away_team_score.max(), 4):
            times = np.linspace(current_point_time, 0, len(prev_df))
            prev_df['times'] = times
            points.append(prev_df)
    GAME_POINTS = pd.concat(points)
    GAME_POINTS.times = GAME_POINTS.times + ((4 - GAME_POINTS.game_quarter)*720)
    try:
        # if error_points > 0:
        #     print(f'{error_points} errors with {GAME_POINTS.gameID.iloc[0]}')
        assert GAME_POINTS.times.is_monotonic_decreasing, f'timing is off somewhere for game: {GAME_POINTS.gameID.iloc[0]}'
        home_team_win = GAME_POINTS[['away_team_score', 'home_team_score']].max().values.argmax() if GAME_POINTS['home_team_score'].max() != GAME_POINTS['away_team_score'].max() else -2
        # home_team_win = pd.Series(home_team_win)
        # home_team_win[home_team_win == 0] = -1
        # home_team_win[home_team_win == -2] = 0
        GAME_POINTS['home_team_win'] = home_team_win
        games.append(GAME_POINTS)
    except AssertionError as e:
        print(e)
        

timing is off somewhere for game: 2022-06-17-SEA-OAK


In [244]:
PROCESSED_GAMES = pd.concat(games).drop(['point_start_time', 'start_on_offense'], axis=1)
PROCESSED_GAMES['thrower_x'] = pd.cut(PROCESSED_GAMES['thrower_x'], bins=5, labels=False)
PROCESSED_GAMES['thrower_y'] = pd.cut(PROCESSED_GAMES['thrower_y'], bins=10, labels=False)

In [246]:
PROCESSED_GAMES[PROCESSED_GAMES.home_team_win >= 0].to_csv('../data/processed/throwing_0627.csv', index = False)

In [245]:
PROCESSED_GAMES

,thrower_x,thrower_y,possession_num,possession_throw,game_quarter,quarter_point,is_home_team,home_team_score,away_team_score,gameID,total_points,times,home_team_win
0,1.0,2.0,1,1,1,0,False,0,0,2021-07-11-MIN-CHI,0,2880.00,0
1,2.0,2.0,1,2,1,0,False,0,0,2021-07-11-MIN-CHI,0,2874.75,0
2,4.0,2.0,1,3,1,0,False,0,0,2021-07-11-MIN-CHI,0,2869.50,0
3,3.0,4.0,1,4,1,0,False,0,0,2021-07-11-MIN-CHI,0,2864.25,0
4,2.0,4.0,1,5,1,0,False,0,0,2021-07-11-MIN-CHI,0,2859.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187355,3.0,6.0,1,5,4,0,False,25,19,2022-07-30-OTT-PHI,44,24.00,1
187356,2.0,5.0,1,6,4,0,False,25,19,2022-07-30-OTT-PHI,44,18.00,1
187357,1.0,5.0,1,7,4,0,False,25,19,2022-07-30-OTT-PHI,44,12.00,1
187358,2.0,5.0,1,8,4,0,False,25,19,2022-07-30-OTT-PHI,44,6.00,1


0          True
1          True
2          True
3          True
4          True
          ...  
187355    False
187356    False
187357    False
187358    False
187359    False
Length: 174795, dtype: bool